# Custom models

As I said, Keras provides great customizability. So you can create not just a custom loss functions, layers but also custom models.

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.utils import plot_model

In [ ]:
input_layer = Input(shape=(1,))
hidden_1 = Dense(30, activation="relu")(input_layer)
hidden_2 = Dense(30, activation="relu")(hidden_1)
output_layer = Dense(1)(hidden_2)
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
plot_model(model, show_shapes=True)

## Subclassing Model class

It is possible to implement this model via subclassing Model class https://keras.io/api/models/model/ 

Inheriting from base Model class lets you use functions like `compile()`, `fit()`, `evaluate()`.

We want to define at least:
* `__init__()` - initialize and set layers
* `call()` - set model architecture for forward pass

We are usually defining layers in `__init__()` function and putting model together in a `call()` function.

In [ ]:
class CustomModel(Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        '''initializes the instance attributes'''
        super().__init__(**kwargs)
        self.hidden1 = Dense(units, activation=activation)
        self.hidden2 = Dense(units, activation=activation)
        self.main_output = Dense(10, activation='softmax')

    def call(self, inputs):
        '''defines the network architecture'''
        cur_input = inputs
        flatten = Flatten()(cur_input)
        hidden1 = self.hidden1(flatten)
        hidden2 = self.hidden2(hidden1)
        # it is possible the reuse the same layers again
        hidden3 = self.hidden2(hidden2)
        cur_output = self.main_output(hidden2)
        
        return cur_output

### Declarative API like Functional or Sequential
**Pros**
* model can be easily saved, cloned, shared
* easy display of structure for display and analysis
* easy to analyse, so framework can be caught before data goes through the model
* model is easier to debug because of the static structure

**Cons**
* its just static 😀
* limited code reuse capability

### Imperative API by subclassing the Model
* easier implementation of loops, conditional branching and other dynamic behavior (you can do anything you want in the call function)
* better code structuring by encapsulating complex blocks of code, reusing of complex code blocks

We do **not** need to create the inputs when subclassing the model. It is just infered during runtime.

Since model's architecture is hidden in within the call method, Keras cannot inspect it.

Keras models can be used just like regular layers in the other model. 

## MNIST demo

In [ ]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
model = CustomModel(units = 64)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, validation_split = 0.2, epochs=10)

In [ ]:
result = model.evaluate(x_test, y_test)
print(f'accuracy: {result[1]*100:.2f}%')

In [ ]:
plot_model(model, show_shapes=True)